In [1]:
upstream = None
product = None

In [2]:
# Parameters
upstream = {"Preprocess features": {"nb": "/Users/bruno/Documents/Hackaton_danone/output/process_data.ipynb", "train_csv": "/Users/bruno/Documents/Hackaton_danone/data/train.csv", "test_csv": "/Users/bruno/Documents/Hackaton_danone/data/test.csv"}}
product = {"nb": "/Users/bruno/Documents/Hackaton_danone/output/train_xgboost.ipynb", "result": "/Users/bruno/Documents/Hackaton_danone/predictions/result_xgboost.json"}


In [3]:
import json
import pickle
import warnings
from pathlib import Path

import numpy as np

In [4]:
import pandas as pd
import xgboost as xgb

warnings.filterwarnings('ignore')

In [5]:
cols = ['is_beverage', 'non_recyclable_and_non_biodegradable_materials_count', 
        'est_co2_agriculture', 'est_co2_consumption', 
        'est_co2_distribution', 'est_co2_packaging', 'est_co2_processing', 
        'est_co2_transportation'] 
target_col = 'ecoscore_grade'

In [6]:
train = pd.read_csv(upstream['Preprocess features']['train_csv'], usecols=cols + [target_col])
test = pd.read_csv(upstream['Preprocess features']['test_csv'], usecols=cols)

In [7]:
X_train, y_train = train[cols], train[target_col]
X_test = test[cols]
for i in cols:
    X_train[i] = X_train[i].astype(float)
    X_test[i] = X_test[i].astype(float)

In [8]:
print(X_train.shape, y_train.shape)
print(X_test.shape)

(20835, 8) (20835,)
(1272, 8)


In [9]:
dtrain = xgb.DMatrix(X_train, label=y_train.values)
dvalid = xgb.DMatrix(X_test)

In [10]:
evals_result = {}
watchlist = [(dtrain, "train")]
params = {
    "seed": 0,
    "eta": 0.1,
    "alpha": 15,
    # "gamma": 25,
    # "lambda": 15,
    "max_depth": 15,
    "refresh_leaf": 1,
    "booster": "gbtree",
    "max_delta_step": 1,
    "n_estimators": 1000,
    # "colsample_bytree": 0.1,
    # "colsample_bylevel": 0.8,
    # "min_child_weight": 11.0,
    # "seed_per_iteration": True,
    'subsample': .1,
    "objective": "multi:softprob",
    'num_class': len(y_train.unique()),
    "eval_metric": ["auc", "mlogloss"],
}
eta_decay = np.linspace(params['eta'], 0.01, 2000).tolist()

In [11]:
model = xgb.train(
    params,
    dtrain,
    params.pop("n_estimators"),
    evals=watchlist,
    verbose_eval=True,
    evals_result=evals_result,
    callbacks=[xgb.callback.LearningRateScheduler(eta_decay)],
)

[0]	train-auc:0.91316	train-mlogloss:1.53056


[1]	train-auc:0.92271	train-mlogloss:1.45774


[2]	train-auc:0.92906	train-mlogloss:1.38904


[3]	train-auc:0.93071	train-mlogloss:1.32450


[4]	train-auc:0.93157	train-mlogloss:1.26424


...

[995]	train-auc:0.96454	train-mlogloss:0.46422


[996]	train-auc:0.96455	train-mlogloss:0.46416


[997]	train-auc:0.96456	train-mlogloss:0.46414


[998]	train-auc:0.96456	train-mlogloss:0.46410


[999]	train-auc:0.96456	train-mlogloss:0.46407


In [12]:
Path(product['result'])\
    .write_text(json.dumps(
        {'target': {index: int(i) 
                    for index, i in enumerate(np.argmax(model.predict(dvalid), axis=1))}
         }))

12894